In [105]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import joblib
import keras

In [10]:
data_set=pd.read_csv("mushroom.csv")
data_set

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Haryana,Ambala,Naraingarh,Mashrooms,Other,13/12/2010,4000,4000,4000
1,Haryana,Ambala,Naraingarh,Mashrooms,Other,14/12/2010,4000,4000,4000
2,Haryana,Ambala,Naraingarh,Mashrooms,Other,29/12/2010,4000,4000,4000
3,Haryana,Ambala,Naraingarh,Mashrooms,Other,31/12/2010,4000,4000,4000
4,Haryana,Ambala,Naraingarh,Mashrooms,Other,01/01/2011,5000,5000,5000
...,...,...,...,...,...,...,...,...,...
5191,Punjab,Ropar (Rupnagar),Chamkaur Sahib,Mashrooms,Other,17/07/2008,1085,1500,1400
5192,Punjab,Sangrur,Dhuri,Mashrooms,Other,01/09/2008,850,850,0
5193,Tamil Nadu,Vellore,Katpadi(Uzhavar Santhai),Mashrooms,Other,26/03/2009,8000,0,0
5194,Uttrakhand,Dehradoon,Rishikesh,Mashrooms,Other,14/12/2012,8000,9000,8500


In [14]:
data_set=data_set.drop(["state","district","commodity","variety","min_price","max_price"],axis=1)

In [15]:
data_set.head()

,market,arrival_date,modal_price
0,Naraingarh,13/12/2010,4000
1,Naraingarh,14/12/2010,4000
2,Naraingarh,29/12/2010,4000
3,Naraingarh,31/12/2010,4000
4,Naraingarh,01/01/2011,5000


In [16]:
data_set["month"]=pd.DatetimeIndex(data_set['arrival_date']).month
data_set["year"]=pd.DatetimeIndex(data_set['arrival_date']).year

In [69]:
data_set.head()

,market,modal_price,month,year
0,Naraingarh,4000,12,2010
1,Naraingarh,4000,12,2010
2,Naraingarh,4000,12,2010
3,Naraingarh,4000,12,2010
4,Naraingarh,5000,1,2011


In [70]:
df=data_set

In [72]:
df.describe(include="all")

,market,modal_price,month,year
count,5196,5196.000000,5196.000000,5196.000000
unique,65,NaN,NaN,NaN
top,Jalandhar City,NaN,NaN,NaN
freq,1539,NaN,NaN,NaN
mean,NaN,7477.466898,6.397614,2009.394919
std,NaN,3144.637460,3.842083,1.910592
min,NaN,0.000000,1.000000,2003.000000
25%,NaN,5100.000000,3.000000,2008.000000
50%,NaN,7500.000000,6.000000,2009.000000
75%,NaN,9500.000000,10.000000,2011.000000


In [73]:
df.shape

(5196, 4)

In [74]:
df=pd.get_dummies(df)
df.head()

,modal_price,month,year,market_Asandh,market_Attabira,market_Badnawar,market_Balugaon,market_Barwala,market_Bhadrak,market_Bhuna,...,market_Rishikesh,market_Roorkee,market_Samalkha,market_Samana,market_Samba,market_Sambalpur,market_Shahdara,market_Sirhind,market_Una,market_kalanwali
0,4000,12,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4000,12,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4000,12,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4000,12,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5000,1,2011,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
y=df["modal_price"]
df=df.drop('modal_price',axis=1)

In [76]:
df.describe(include="all")

,month,year,market_Asandh,market_Attabira,market_Badnawar,market_Balugaon,market_Barwala,market_Bhadrak,market_Bhuna,market_Bhuntar,...,market_Rishikesh,market_Roorkee,market_Samalkha,market_Samana,market_Samba,market_Sambalpur,market_Shahdara,market_Sirhind,market_Una,market_kalanwali
count,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,...,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000,5196.000000
mean,6.397614,2009.394919,0.000770,0.002502,0.000385,0.004619,0.000577,0.000192,0.000385,0.000192,...,0.000192,0.000192,0.000577,0.005581,0.000192,0.009045,0.092571,0.000192,0.000962,0.000577
std,3.842083,1.910592,0.027738,0.049961,0.019617,0.067812,0.024024,0.013873,0.019617,0.013873,...,0.013873,0.013873,0.024024,0.074506,0.013873,0.094685,0.289858,0.013873,0.031009,0.024024
min,1.000000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,2009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.000000,2011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12.000000,2012.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
y.head()

0    4000
1    4000
2    4000
3    4000
4    5000
Name: modal_price, dtype: int64

In [26]:
data_set=data_set.drop("arrival_date",axis=1)

In [28]:
data_set.dtypes

market         object
modal_price     int64
month           int64
year            int64
dtype: object

In [80]:
X_train,X_test,y_train,y_test=train_test_split(df,y,test_size=0.2)

In [82]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json 

In [126]:
model=Sequential()
model.add(Dense(units=2,input_dim=67,activation='relu'))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(200, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(50, activation="relu"))
model.add(Dense(units=1,activation='linear'))
model.compile(loss='mean_absolute_error',optimizer="adam",metrics=['mean_absolute_error',"accuracy"])

In [127]:
model_fit=model.fit(X_train,y_train,batch_size=50,epochs=50)

Epoch 1/50
84/84 [==============================] - 0s 3ms/step - loss: 4328.6973 - mean_absolute_error: 4328.6978 - accuracy: 0.0000e+00
Epoch 2/50
84/84 [==============================] - 0s 3ms/step - loss: 2460.0400 - mean_absolute_error: 2460.0400 - accuracy: 0.0000e+00: 0s - loss: 2414.8591 - mean_absolute_error: 2414.8591 - accuracy: 
Epoch 3/50
84/84 [==============================] - 0s 3ms/step - loss: 2466.6082 - mean_absolute_error: 2466.6082 - accuracy: 0.0000e+00
Epoch 4/50
84/84 [==============================] - 0s 4ms/step - loss: 2463.9580 - mean_absolute_error: 2463.9580 - accuracy: 0.0000e+00
Epoch 5/50
84/84 [==============================] - 0s 3ms/step - loss: 2462.7290 - mean_absolute_error: 2462.7290 - accuracy: 0.0000e+00
Epoch 6/50
84/84 [==============================] - 0s 3ms/step - loss: 2473.0974 - mean_absolute_error: 2473.0974 - accuracy: 0.0000e+00
Epoch 7/50
84/84 [==============================] - ETA: 0s - loss: 2459.7351 - mean_absolute_error: 245

In [94]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
rfr.score(X_test,y_test)

0.6541507489617197

In [139]:
rfr.predict([4,2010,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,])

ValueError: Expected 2D array, got 1D array instead:
array=[4.00e+00 2.01e+03 1.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [96]:
from sklearn.metrics import r2_score
from sklearn import tree

clf=tree.DecisionTreeRegressor()
clf.fit(X_train, y_train)
x_predicted=clf.predict(X_test)
r2_score(x_predicted,y_test)

0.4924197737929539

In [128]:
x_predicted=model.predict(X_test)

In [157]:
import numpy as np
x=np.array([[4,2010,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]])
model.predict(x)

array([[6986.5405]], dtype=float32)

In [159]:
x.size

67

In [135]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 2)                 136       
_________________________________________________________________
dense_61 (Dense)             (None, 300)               900       
_________________________________________________________________
dense_62 (Dense)             (None, 200)               60200     
_________________________________________________________________
dense_63 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_64 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 51        
Total params: 86,437
Trainable params: 86,437
Non-trainable params: 0
_________________________________________________

In [136]:
model_json=model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

In [154]:
model.save_weights("model.h5")
model.save("mshroom_model.h5")
print("Model succesfully saved")

Model succesfully saved


In [138]:
import joblib
joblib.dump(rfr, "mushroom_model.pkl")

my_model_loaded = joblib.load("mushroom_model.pkl")

In [143]:
market=df.columns.tolist()

In [145]:
market=market[2:]
market

['market_Asandh',
 'market_Attabira',
 'market_Badnawar',
 'market_Balugaon',
 'market_Barwala',
 'market_Bhadrak',
 'market_Bhuna',
 'market_Bhuntar',
 'market_Chamba',
 'market_Chamkaur Sahib',
 'market_Cherthalai',
 'market_Dhanotu (Mandi)',
 'market_Dharamkot',
 'market_Dhenkanal',
 'market_Dhuri',
 'market_Digapahandi',
 'market_Dudhansadhan',
 'market_Fatehabad',
 'market_Firozepur City',
 'market_Garobadha',
 'market_Godabhaga',
 'market_Gosala',
 'market_Gunpur',
 'market_Gunupur(Maniguda)',
 'market_Gurdaspur',
 'market_Hamirpur',
 'market_Jalandhar City',
 'market_Kangra',
 'market_Kangra(Baijnath)',
 'market_Kangra(Jassour)',
 'market_Kangra(Nagrota Bagwan)',
 'market_Kantabaji',
 'market_Kathua',
 'market_Katpadi(Uzhavar Santhai)',
 'market_Kesinga',
 'market_Kot ise Khan',
 'market_Kottarakka',
 'market_Machhiwara',
 'market_Malout',
 'market_Mohindergarh',
 'market_Nahan',
 'market_Nakodar',
 'market_Naraingarh',
 'market_Narwal Jammu (F&V)',
 'market_Nawan Shahar(Subzi M

In [148]:
x=[a.strip('market_') for a in market]

In [149]:
x

['Asandh',
 'Attabi',
 'Badnaw',
 'Balugaon',
 'Barwal',
 'Bhad',
 'Bhun',
 'Bhun',
 'Chamb',
 'Chamkaur Sahib',
 'Cherthalai',
 'Dhanotu (Mandi)',
 'Dharamko',
 'Dhenkanal',
 'Dhuri',
 'Digapahandi',
 'Dudhansadhan',
 'Fatehabad',
 'Firozepur City',
 'Garobadh',
 'Godabhag',
 'Gosal',
 'Gunpu',
 'Gunupur(Maniguda)',
 'Gurdaspu',
 'Hamirpu',
 'Jalandhar City',
 'Kang',
 'Kangra(Baijnath)',
 'Kangra(Jassour)',
 'Kangra(Nagrota Bagwan)',
 'Kantabaji',
 'Kathu',
 'Katpadi(Uzhavar Santhai)',
 'Kesing',
 'Kot ise Khan',
 'Ko',
 'Machhiw',
 'Malou',
 'Mohindergarh',
 'Nahan',
 'Nakod',
 'Naraingarh',
 'Narwal Jammu (F&V)',
 'Nawan Shahar(Subzi Mandi)',
 'Nawanshah',
 'Nobarangpu',
 'Palampu',
 'Paonta Sahib',
 'Pataudi',
 'Pathanko',
 'Phillaur(Apra Mandi)',
 'Quadian',
 'Rajouri (F&V)',
 'Rajpu',
 'Rishikesh',
 'Roo',
 'Samalkh',
 'Saman',
 'Samb',
 'Sambalpu',
 'Shahd',
 'Sirhind',
 'Un',
 'lanwali']

In [153]:
len(x)

65